In [3]:
from scipy.spatial import distance as dist
from imutils import face_utils
import numpy as np
import datetime
import imutils
import time
import dlib
import cv2

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/home/amit/driver1/facial-landmarks/shape_predictor_68_face_landmarks.dat')

In [5]:
    gray = cv2.imread('user.1.jpg',0)
    
    rects = detector(gray, 0)
    
    def eye_aspect_ratio(eye):

        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        
            
        for (name, (j, k)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            (x, y, w, h) = cv2.boundingRect(np.array([shape[j:k]]))
            roi = gray[y-10:y + h+10, x-10:x + w+10]
            #roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            if(j==36):
                right_eye = roi
            elif(j==42):
                left_eye = roi
                
     
        
    def finding_pupil_centre(right_eye, ear):
        equ = cv2.equalizeHist(right_eye)
        hist, bin_edges = np.histogram(equ.flatten(), 256, [0,256])
        pupil_pixels = equ.size*0.08
        threshold_fake = 0
        for i in range(0, 256):
            threshold_fake = threshold_fake+hist[i]
            if(threshold_fake>pupil_pixels):
                break
        
        threshold1 = i
    
        new_img = np.zeros((equ.shape[0],equ.shape[1]))
        for i in range(5, equ.shape[0]-5):
            for j in range(5, equ.shape[1]-5):
                if equ[i,j] < threshold1:
                    new_img[i,j] = 255
                else:
                    new_img[i,j] = 0
                
        kernel = np.ones((2,2), np.uint8)
        new_img_er1 = cv2.erode(new_img, kernel, iterations=1)           
    
        new_img_er1 = np.uint8(new_img_er1)
        im2, contours, hierarchy = cv2.findContours(new_img_er1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        if(not contours):
            return right_eye
        
        right_eye2 = right_eye
        c = max(contours, key = cv2.contourArea)
        M = cv2.moments(c)

        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX, cY = 0, 0
            
        if(ear<0.2):
            cX, cY = 0, 0    
        cv2.circle(right_eye2, (cX, cY), 2, (255, 255, 255), -1)
        
        return right_eye2
    
    new_right_eye = finding_pupil_centre(right_eye, ear)
    new_left_eye = finding_pupil_centre(left_eye, ear)
    cv2.imwrite('user.1.(right_eye_pupil).jpg', new_right_eye)
    cv2.imwrite('user.1.(left_eye_pupil).jpg', new_left_eye)

True